In [1]:
import os

api_key = os.environ.get("GEMINI_API_KEY")

In [2]:
!pip install google-generativeai
!pip install google-api-core


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [ ]:
import os
import json
import time 
import google.generativeai as genai
from google.api_core.exceptions import ResourceExhausted


SAFETY_CONFIG = [
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
]


def load_model(api_key):
    """Initialize and return the Gemini model with the provided API key."""
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel("gemini-2.0-flash")
    return model


def get_recommendations(request, model, api_key, prompt):
    """
    Generate recommendations based on the request.

    Args:
        request: A dictionary containing 'entity', 'genre', and optionally 'example'
        model: The Gemini model to use
        api_key: The API key for fallback in case of rate limiting

    Returns:
        A tuple of (recommendations_list, raw_response)
    """
    entity = request.get("entity", "movies")
    genre = request.get("genre", "")

    prompt = prompt.replace("{entity}", entity)
    prompt = prompt.replace("{genre}", genre)

    def do_request():
        try:
            chat_completion = model.generate_content(
                prompt, safety_settings=SAFETY_CONFIG
            )

            if (
                str(chat_completion.prompt_feedback).strip()
                == "block_reason: OTHER".strip()
            ):
                return None, "Content was blocked due to safety settings."

            response_text = chat_completion.text

            if "```json" in response_text:
                json_content = response_text.split("```json")[1].split("```")[0].strip()
            elif "```" in response_text:
                json_content = response_text.split("```")[1].split("```")[0].strip()
            else:
                json_content = response_text.strip()

            try:
                recommendations = json.loads(json_content)
                if not isinstance(recommendations, list):
                    return None, f"Invalid response format: {response_text}"

                for item in recommendations:
                    if not all(key in item for key in ["name", "language", "country"]):
                        return (
                            None,
                            f"Missing required fields in response: {response_text}",
                        )

                time.sleep(0.5)
                return recommendations, response_text
            except json.JSONDecodeError:
                return None, f"Failed to parse JSON response: {response_text}"

        except Exception as e:
            return None, f"Error generating recommendations: {str(e)}"

    try:
        return do_request()
    except ResourceExhausted:
        print(f"Rate limit exceeded for API key: {api_key}")
        time.sleep(60)  
        return do_request()
    except Exception as e:
        print(f"Unexpected error: {str(e)}")
        return None, str(e)


def recommender(lang: str, version: str):
    """Main function to process recommendation requests."""

    with open("prompts.json", "r") as f:
        prompts = json.load(f)

    
    if lang == "persian" and version == "v2":
        prompt = prompts[lang + "_v2"]
    
    else:
        prompt = prompts[lang]
        prompt_addition = "" if version == "v1" else prompts[lang + "_v2"]
        prompt = prompt + prompt_addition

    

    genres = [
        "Docu",
        "Action",
        "Drama",
        "Horror",
        "Fantasy",
        "Romance",
        "Mystery",
        "Thriller",
        "Comedy",
        "Sci-Fi",
        "",
    ]

    example_requests = [{"entity": "movies", "genre": genre} for genre in genres]
    model = load_model(api_key)
    output_dir = f"{lang}_recommendation_output_{version}/"
    os.makedirs(output_dir, exist_ok=True)
    for experiment_id in range(5):
        
        experiment_dir = os.path.join(output_dir, f"experiment_{experiment_id}")
        os.makedirs(experiment_dir, exist_ok=True)
        for i, request in enumerate(example_requests):
            request_id = f"request_{i}"
            if os.path.exists(os.path.join(experiment_dir, f"recommendations_{request_id}.json")):
                print(f"Skipping request {request_id} as it already exists.")
                continue
            
            example_requests[i]["request_id"] = request_id
            recommendations, raw_response = get_recommendations(request, model, api_key, prompt)

            if recommendations is None or "Error" in raw_response:
                print(f"Skipping request {request_id} due to error: {raw_response}")
                #wait 3 mins cause of rate limit
                print("Waiting for 3 minutes before retrying...")
                time.sleep(180)
                continue


            result = {
                "request_id": request_id,
                "request": request,
                "recommendations": recommendations,
                "raw_response": raw_response,
            }
            with open(
                os.path.join(experiment_dir, f"recommendations_{request_id}.json"), "a"
            ) as f:
                f.write(json.dumps(result) + "\n")
            
            print(f"Processed request {request_id} for {request['entity']} in {request['genre']} genre.")

In [12]:
for lang in ["english", "persian", "turkish", "italian"]:
    recommender(lang, "v2")

Skipping request request_0 as it already exists.
Skipping request request_1 as it already exists.
Skipping request request_2 as it already exists.
Skipping request request_3 as it already exists.
Skipping request request_4 as it already exists.
Skipping request request_5 as it already exists.
Skipping request request_6 as it already exists.
Skipping request request_7 as it already exists.
Skipping request request_8 as it already exists.
Skipping request request_9 as it already exists.
Skipping request request_10 as it already exists.
Skipping request request_0 as it already exists.
Skipping request request_1 as it already exists.
Skipping request request_2 as it already exists.
Skipping request request_3 as it already exists.
Skipping request request_4 as it already exists.
Skipping request request_5 as it already exists.
Skipping request request_6 as it already exists.
Skipping request request_7 as it already exists.
Skipping request request_8 as it already exists.
Skipping request re